# 변동성 조절 전략(Target Volatility Strategies) 포트폴리오 산출 기본


In [0]:
import numpy as np
import pandas as pd
import scipy.optimize as sco
import datetime as dt
from datetime import timedelta
from pylab import *

# 예스트레이더 출력 결과물을 Pandas 형태로 포맷 변환

In [0]:
# 예스트레이더 종목검색 결과를 날짜 기준으로 포맷팅
def convert_yes_price_file(gubun):
    # 우리나라 시간 = 그리니치 표준시 + 9시
    file_dt = (dt.datetime.now() + dt.timedelta(hours=9)).strftime('%Y%m%d')

    yes_price_file = pd.DataFrame()
    # 예스트레이더 출력용 지표식과 칼럼수 일치시켜야 함.
    yes_price_file = pd.read_csv('yes_stock_price_' + file_dt + '.csv',encoding= 'euc-kr',
                           names = ['날짜1','종목명','종목코드','날짜','시가','고가','저가','종가','거래량',
                                    '단기이평','중기이평','장기이평','지수단기이평','지수중기이평','지수장기이평',
                                    'MACD','MACD_OSC','ATR','CCI','StoK','StoD','TRIX','이격도'],
                           index_col= 1, header=None, dtype={'종목코드':str})

    종목명 = yes_price_file.index.unique()
    종목코드 = yes_price_file['종목코드'].unique()  # 종목코드 사용할 경우를 위해 별도 저장
    df1 = {'종목명':종목명,
           '종목코드':종목코드}
    df1 = pd.DataFrame(df1)
    df1.to_csv('symbol_cd.csv')

    cls_p_data = pd.DataFrame()
    vol_data = pd.DataFrame()
    if gubun == 1:
        hi_p_data = pd.DataFrame()
        lo_p_data = pd.DataFrame()
        sma_data = pd.DataFrame()
        mma_data = pd.DataFrame()
        lma_data = pd.DataFrame()
        sema_data = pd.DataFrame()
        mema_data = pd.DataFrame()
        lema_data = pd.DataFrame()
        macd_data = pd.DataFrame()
        macdo_data = pd.DataFrame()
        atr_data = pd.DataFrame()
        cci_data = pd.DataFrame()
        stok_data = pd.DataFrame()
        stod_data = pd.DataFrame()
        trix_data = pd.DataFrame()
        dis_data = pd.DataFrame()

    for i in range(len(종목명)):
        cls_p = yes_price_file.loc[종목명[i],['날짜','종가']]
        cls_p.set_index(['날짜'],inplace=True)
        cls_p_data = pd.concat([cls_p_data,cls_p],axis=1) 

        vol = yes_price_file.loc[종목명[i],['날짜','거래량']]
        vol.set_index(['날짜'],inplace=True)
        vol_data = pd.concat([vol_data,vol],axis=1) 

        if gubun == 1:
            hi_p = yes_price_file.loc[종목명[i],['날짜','고가']]
            hi_p.set_index(['날짜'],inplace=True)
            hi_p_data = pd.concat([hi_p_data,hi_p],axis=1) 

            lo_p = yes_price_file.loc[종목명[i],['날짜','저가']]
            lo_p.set_index(['날짜'],inplace=True)
            lo_p_data = pd.concat([lo_p_data,lo_p],axis=1) 

            sma = yes_price_file.loc[종목명[i],['날짜','단기이평']]
            sma.set_index(['날짜'],inplace=True)
            sma_data = pd.concat([sma_data,sma],axis=1) 

            mma = yes_price_file.loc[종목명[i],['날짜','중기이평']]
            mma.set_index(['날짜'],inplace=True)
            mma_data = pd.concat([mma_data,mma],axis=1) 

            lma = yes_price_file.loc[종목명[i],['날짜','장기이평']]
            lma.set_index(['날짜'],inplace=True)
            lma_data = pd.concat([lma_data,lma],axis=1) 

            atr = yes_price_file.loc[종목명[i],['날짜','ATR']]
            atr.set_index(['날짜'],inplace=True)
            atr_data = pd.concat([atr_data,atr],axis=1) 

    cls_p_data.columns = 종목명
    print('\n 종가데이터 \n', cls_p_data.head())
    cls_p_data.to_csv('cls_p_data.csv')

    vol_data.columns = 종목명
    print('\n 거래량 \n', vol_data.head())
    vol_data.to_csv('volume_data.csv')

    if gubun == 1:
        hi_p_data.columns = 종목명
        print('\n 고가데이터 \n', hi_p_data.head())
        hi_p_data.to_csv('hi_p_data.csv')

        lo_p_data.columns = 종목명
        print('\n 저가데이터 \n', lo_p_data.head())
        lo_p_data.to_csv('lo_p_data.csv')

        sma_data.columns = 종목명
        print('\n 단기이평 \n', sma_data.head())
        sma_data.to_csv('sma_data.csv')

        mma_data.columns = 종목명
        print('\n 중기이평 \n', mma_data.head())
        mma_data.to_csv('mma_data.csv')

        lma_data.columns = 종목명
        print('\n 장기이평 \n', lma_data.head())
        lma_data.to_csv('lma_data.csv')

        atr_data.columns = 종목명
        print('\n ATR \n', atr_data.head())
        atr_data.to_csv('atr_data.csv')

In [0]:
gubun = 0 # 지표 파일을 만들려면 1로, 안 만들여면 0
convert_yes_price_file(gubun)  

# 사용자 함수 선언

In [0]:
def statistics(w,r,c):
    '''포트폴리오 총계치 출력
    인수
    =====
    weight : array-like  포트폴리오 내의 비중
    
    반환값
    ======
    portfolio_return     : float 포트폴리오 수익률 기댓값
    portfolio_volatility : float 포트폴리오 변동성 기댓값
    sharpe_ratio         : float 무위험 이자율이 0일 때의 샤프 지수
    '''
    w = np.array(w)
    portfolio_return = np.sum(r * w)
    portfolio_volatility = np.sqrt(np.dot(w.T,np.dot(c, w)))
    sharp_ratio = portfolio_return / portfolio_volatility
    return np.array([portfolio_return, portfolio_volatility, sharp_ratio])

In [0]:
def min_func_sharpe(w,r,c):
# 최소화문제에서 샤프지수 최대값을 찾으려면 샤프지수의 음수 값을 최소화하면 된다.
    return -statistics(w,r,c)[2]  # 위에서 만든 statistics의 3번째 값이 샤프지수

In [0]:
def min_func_variance(w,r,c):
# 분산 최소화 함수
    return statistics(w,r,c)[1]**2

In [0]:
# 기대수익률 음수값의 최소화 함수
def min_func_return(w,r,c):
    return -statistics(w,r,c)[0]

In [0]:
def solve_target_vol(r,c,t):
    noa = len(r)
    def port_volatility(w):
        return np.sqrt(np.dot(w.T,np.dot(c, w)))

    # 제약조건
    cons = ({'type': 'eq', 'fun': lambda x: port_volatility(x) - t},
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
            {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 4},   #매수 종목이 5 종목 이상되도록 조건 설정 
            {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 3},
            {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 2},
            {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 1}) 

    # 범위값
    bnds = tuple((0, 1) for x in range(noa))

    opts = sco.minimize(min_func_return, noa * [1. / noa, ], (R, C), method='SLSQP', bounds=bnds, constraints=cons)
    return opts.x

In [0]:
def solve_target_vol2(r,c,t):
    noa = len(r)
    def port_volatility(w):
        return np.sqrt(np.dot(w.T,np.dot(c, w)))

    # 제약조건
    cons = ({'type': 'eq', 'fun': lambda x: port_volatility(x) - t},
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 

    # 범위값
    bnds = tuple((0, 1) for x in range(noa))

    opts = sco.minimize(min_func_return, noa * [1. / noa, ], (R, C), method='SLSQP', bounds=bnds, constraints=cons)
    return opts.x

In [0]:
def solve_frontier(r,c):
    def min_func_port(w,r,c):
        return statistics(w,r,c)[1]  # 두번째 값 변동성의 최소화
    def port_return(w):
        return np.sum(r * w)
    mean, vol = [], []
    for t in np.linspace(min(r), max(r), num=50): # Iterate through the range of returns on Y axis
        # 제약조건
        cons = ({'type': 'eq', 'fun': lambda x:  port_return(x) - t},
                {'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})
        # 범위값
        bnds = tuple((0, 1) for x in range(noa))
        opts = sco.minimize(min_func_port, noa * [1. / noa, ],(r,c), method='SLSQP', constraints=cons, bounds=bnds)
        if not opts.success:
            raise BaseException(opts.message)
        # add point to the min-var frontier [x,y] = [opts.x, t]
        mean.append(t)                                                 # return
        vol.append(statistics(opts['x'],r,c)[1])   # min-variance based on optimized weights

    return array(mean), array(vol)

In [0]:
def display_Efficient_frontier(r,c):
    mean, vol = solve_frontier(r,c)
    plot(vol, mean, color='black'), grid(True)  # draw efficient frontier
    print('최소 vol = ', min(vol).round(3), ', 최대 vol = ', max(vol).round(3))

# Main

In [0]:
공분산_산출기간 = 500  # 시뮬레이터를 통해 최적 값 찾을 것

raw_data = pd.read_csv('cls_p_data.csv',index_col=['날짜'], parse_dates=['날짜'])
종목명 = raw_data.columns
print('\n >> 종목명 :', 종목명)

noa = len(종목명)

symbol_data = pd.read_csv('symbol_cd.csv',index_col=None,dtype={'종목코드':str})
종목코드 = symbol_data['종목코드']
print('\n >> 종목코드 :', 종목코드)

work_data = raw_data.iloc[-공분산_산출기간:]
현재가 = list(raw_data.iloc[-1])
print('\n >> 현재가 :', 현재가)

로그수익률 = np.log(work_data / work_data.shift(1))
print(로그수익률.head())

R = 로그수익률.mean() * 252  # 연수익률
print(R)

C = 로그수익률.cov() * 252 # 공분산
print(C)

# 우리나라 시간 = 그리니치 표준시 + 9시
current_dt = (dt.datetime.now() + timedelta(hours=9)).strftime('%Y%m%d')
date_col = [current_dt for i in range(noa)] # 날짜 칼럼 생성
print(date_col)

In [0]:
display_Efficient_frontier(R,C)


# 목표 변동성 입력

In [0]:
# 투자성향별 목표표준편차, 상단의 그림을 참조해서 변경 입력한다.
target_vol = (0.4, 0.5, 0.6)

# 목표 변동성별 모델포트폴리오 산출

In [0]:
display_Efficient_frontier(R,C)

for i in range(len(target_vol)):
    # 최소 5종목 조건 없음
    W = solve_target_vol2(R,C,target_vol[i])
#    print(W)
    stat = statistics(W,R,C).round(3)  # 샤프지수 최대 포트폴리오의 수익률, 표준편차, 샤프지수
#    print('\n>> 수익률 = ', stat[0],', 표준편차 = ', stat[1], ', 샤프지수 = ', stat[2])

    scatter(stat[1], stat[0], marker='o', color='red'), grid(True)

    mp = {'날짜':date_col,
          '종목명':종목명,
          '종목코드':종목코드,
          '비중':W.round(3),
          '현재가':현재가}
    df1 = pd.DataFrame(mp,columns = ['날짜','종목명','종목코드','비중','현재가'])
#    print(df1)
    df1.to_csv('TV_model_portfolio_'+str(i+1)+'.csv')

최소 보유종목(5종목 이상) 조건 반영

In [0]:
display_Efficient_frontier(R,C)

for i in range(len(target_vol)):
    # 최소 보유종목 조건 반영
    W = solve_target_vol(R,C,target_vol[i])
#    print(W)
    stat = statistics(W,R,C).round(3)  # 샤프지수 최대 포트폴리오의 수익률, 표준편차, 샤프지수
#    print('\n>> 수익률 = ', stat[0],', 표준편차 = ', stat[1], ', 샤프지수 = ', stat[2])

    scatter(stat[1], stat[0], marker='o', color='red'), grid(True)

    mp = {'날짜':date_col,
          '종목명':종목명,
          '종목코드':종목코드,
          '비중':W.round(3),
          '현재가':현재가}
    df1 = pd.DataFrame(mp,columns = ['날짜','종목명','종목코드','비중','현재가'])
#    print(df1)
    df1.to_csv('TV_model_portfolio_'+str(i+1)+'.csv')